In [2]:
import os
import numpy as np
import pandas as pd
import folium
import json
dirpath = os.getcwd()
print('dirpath', dirpath)

dirpath /Users/carlosvaldez/Documents/data-vis-mx


In [54]:
JANUARY = 9
DECEMBER = 21
# Create Query search class
class QuerySearch():
    def __init__(self, year=None, startMonthIdx=JANUARY, endMonthIdx=DECEMBER):
        self.data = data
        self.startMonthIdx = startMonthIdx
        self.endMonthIdx = endMonthIdx
        self.year = year # default to the whole thing if nothing is passed in
        
    def checkYear(self, series, year):
        if(series['Año'] == year):
            return sumDate(series)
        else:
            return 0

    def checkTypeOfCrime(self, series, crime):
        if(series['Tipo de delito'] == crime):
            return sumDate(series)
        else:
            return 0
    
    def checkGender(self, series, crime, gender):
        if(series['Tipo de delito'] == crime and series['Sexo'] == gender):
            return sumDate(series)
        else:
            return 0
    
    def sumDate(self, s):
        result = 0
        if(self.year is not None and s['Año'] == self.year):
            for i in range(self.startMonthIdx, self.endMonthIdx):
                result += s[i]
#         elif(s['Año'] != self.year):
#             return result
        else:
            for i in range(self.startMonthIdx, self.endMonthIdx):
                result += s[i]
            return result
        

In [55]:
data = pd.read_csv('delitos.csv', encoding="latin_1")

def total_abs(data, crimeType, year=None, startMonthIdx=JANUARY, endMonthIdx=DECEMBER):
    query = QuerySearch(startMonthIdx=startMonthIdx, endMonthIdx=endMonthIdx)
    d = {} # dictionary!
    for index, series in data.iterrows():
        if(series['Entidad'] in d):
            curr = d[series['Entidad']] # the tuple curr[0] == total, curr[1] == homicidios totales
            d[series['Entidad']] = (curr[0], 
                                    curr[1] + query.sumDate(series), 
                                    curr[2] + query.checkTypeOfCrime(series, crimeType),
                                    curr[3] + query.checkGender(series, crimeType, "Hombre"),
                                    curr[4] + query.checkGender(series, crimeType, "Mujer"),
                                    curr[5] + query.checkGender(series, crimeType, "No identificado"))
        else:
            d[series['Entidad']] = (series['Entidad'], 
                                    query.sumDate(series), 
                                    query.checkTypeOfCrime(series, crimeType),
                                    query.checkGender(series, crimeType, "Hombre"),
                                    query.checkGender(series, crimeType, "Mujer"),
                                    query.checkGender(series, crimeType, "No identificado"))        
    state_results = pd.DataFrame.from_dict(d, orient='index', columns=['Estado', 'Total', 'Numero de ' + crimeType + 's', 'Hombre', 'Mujer', 'No identificado'])
    l = [num for num in range(1, 33)]
    state_results.index = l
    # return data frame.
    return state_results

total_abs(data, "Homicidio")


NameError: name 'sumDate' is not defined

In [7]:
# Homicides over 2015 - 2019 mapped. 
centro_lat, centro_lon = 22.396092, -101.731430
m = folium.Map(location=[centro_lat, centro_lon], zoom_start=5, tiles='cartodb positron')
folium.Choropleth(
    geo_data=open("statesGEOJSON.json",encoding = "utf-8-sig").read(),
    name='Mexico',
    highlight=True,
    data=state_results,
    columns=['Estado', 'Homicidios'],
    key_on='feature.properties.NOM_ENT',
    fill_color='RdPu',
    nan_fill_color='Gray',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Estados'
).add_to(m)

m.save('homicides.html')

In [ ]:
# GIF of Homicides year by year.... 
